In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from skimage import img_as_ubyte, io, color, exposure
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import os
import cv2


In [ ]:
file_path = r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF\CH\Macassa_clearance\Box3\AdaptiveZ_10mm\Core_000_Box_003_of_020_Part_2_of_4"
img_path = os.path.join(file_path, "LIDAR_001_intensity_preview.png")

In [ ]:

img = io.imread(img_path)

if img.dtype != np.uint16:
    img = img.astype(np.uint16)

height, width = img.shape

y, x = np.indices((height, width))

x_flat = x.flatten()
y_flat = y.flatten()
values_flat = img.flatten()

point_cloud2 = np.vstack((x_flat, y_flat, values_flat)).T


In [ ]:
def get_glcm_properties(image, start = (0,0), end = (0,0), distances=[1,2,3,5,10,15,20], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], properties=['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation'], distance_id=0, angle_id=0):
    
    x_start = start[0]
    y_start = start[1]

    x_end = end[0]
    y_end = end[1]
    
    indices = np.where((point_cloud[:, 0] >= x_start) & 
                       (point_cloud[:, 0] <= x_end) & 
                       (point_cloud[:, 1] >= y_start) & 
                       (point_cloud[:, 1] <= y_end))[0]
    
    x_values = np.unique(point_cloud[indices,0])
    y_values = np.unique(point_cloud[indices,1])

    z_values = cv2.normalize(point_cloud[:, 2], None, 0, 1, cv2.NORM_MINMAX)
    z_values = z_values[indices]

    min_z = np.min(z_values)
    max_z = np.max(z_values)

    z_values = np.reshape(z_values,(len(y_values),len(x_values)))


    img_ubyte = img_as_ubyte(z_values)

    glcm = graycomatrix(img_ubyte, distances=distances, angles=angles, symmetric=True, normed=True)

    props_matrix = []
    for distance_id in range(len(distances)):
        for angle_id in range(len(angles)):
            props_vector = []
            props = [graycoprops(glcm, prop)[distance_id, angle_id] for prop in properties]
            props_vector.extend(props)
            # Add the vector to the matrix
            props_matrix.append(props_vector)

    return np.array(props_matrix)

In [ ]:

coords_file = os.path.join(file_path, '.component_parameters.txt')
with open(coords_file) as file:
        lines = file.readlines()
            
        for line in lines:
            if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
            elif "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
            elif "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())

In [ ]:

point_cloud = []

if os.path.isdir(file_path):
    lidar_files = [fn for fn in os.listdir(
        file_path) if fn.endswith('.bpc')]
    if any(lidar_files):
        lidar_filename = file_path + os.sep + lidar_files[0]

data = np.fromfile(lidar_filename, dtype=np.float32)
point_cloud = data.reshape(-1, 3)  # to xyz

ff = ~np.isnan(point_cloud).any(axis=1)
point_cloud = point_cloud[ff, ...]

point_cloud[:, 1] = point_cloud[:, 1] - float(y_offset)

print(f"{file_path} is loaded. \n# of point {point_cloud.shape[0]}")

In [ ]:

matrix_file = (os.path.join(file_path, ".XRAY_DPP_001.lidar2xrf"))
with open(matrix_file, 'r') as file:
    lines = file.readlines()

transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

num_points = point_cloud.shape[0]

homogeneous_points = np.hstack((point_cloud, np.ones((num_points, 1))))
transformed_points = homogeneous_points @ transformation_matrix.T
point_cloud_transformed = transformed_points[:, :3]
point_cloud = point_cloud_transformed
point_cloud = point_cloud[(point_cloud[:,0] >= x_stop) & (point_cloud[:,0] <= x_start)]


print(f"# of point {point_cloud.shape[0]}")

In [ ]:
def print_properties(data,point_1,point_2):
    props = get_glcm_properties(data,point_1,point_2)
    properties = {'contrast': 0, 'dissimilarity': 0, 'homogeneity': 0, 'energy': 0, 'correlation': 0}

    for i in props:
        properties['contrast'] += i[0]
        properties['dissimilarity'] += i[1]
        properties['homogeneity'] += i[2]
        properties['energy'] += i[3]
        properties['correlation'] += i[4]

    num_props = len(props)
    averages = {key: value / num_props for key, value in properties.items()}

    for key, value in averages.items():
        print(f"{key.capitalize()}: {value:.2f}")
        

In [ ]:
%matplotlib qt

# Flatten the point cloud
x = point_cloud[:, 0]
y = point_cloud[:, 1]
z = point_cloud[:, 2]

# Randomly downsample the data with a larger sample size
downsampled_indices = np.random.choice(point_cloud.shape[0], size=100000, replace=False)
x_downsampled = x[downsampled_indices]
y_downsampled = y[downsampled_indices]
z_downsampled = z[downsampled_indices]

# Create a scatter plot with a heat map for downsampled data
fig, ax = plt.subplots(figsize=(10, 8))  # Define `fig` and `ax` for the plot
scatter = ax.scatter(x_downsampled, y_downsampled, c=z_downsampled, cmap='binary', s=1) #flag
plt.colorbar(scatter, label='Z value')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Point Cloud Heat Map (Downsampled)')


plt.ion()
plt.show()


# Function to be called when the rectangle is selected
def onselect(eclick, erelease):
    start = (eclick.xdata, eclick.ydata)
    end = (erelease.xdata, erelease.ydata)
    print_properties(point_cloud,start,end)
    

# Create a RectangleSelector
rect_selector = RectangleSelector(
    ax,
    onselect,
    interactive=True,
    drag_from_anywhere=True,
    button=[1],  # Left mouse button
    minspanx=5,
    minspany=5,
    spancoords='data'
)

